# Olist review_score threshold

Our goal is to help Olist find out what sellers that shall be seen as not valuable

We will do this by implementing a review_score threshold for sellers. Below that threshold, sellers get removed from the platform.

Revenue
As we have detailed previously, Olist charges sellers with various fees. For simplicity, we will assume that Olist takes 20% of the booking_value of sellers and charge 80 BRL by month per seller.

👉 Note: The booking_value is defined as the sum of product price. It excludes freight_value.

Cost

On the long term, bad customer experience has business implications: low repeat rate, immediate customer support cost, refund or non favorable word of mouth.

We will assume that we have an estimate measure of the monetary cost (in Brazilian Real) for each review

In [124]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact
from datetime import datetime
from datetime import datetime, timedelta



#import our classes from the olist dataset
from olist.data import Olist
from olist.order import Order
from olist.product import Product
from olist.seller import Seller

In [2]:
sellers = Seller().get_training_data()
orders = Olist().get_data()['olist_orders_dataset']
orders_details = Olist().get_data()['olist_order_items_dataset']
reviews = Olist().get_data()['olist_order_reviews_dataset']

We will assume that we have an estimate measure of the monetary cost (in Brazilian Real) for each review:

In [3]:
df_list = [[1,100],[2,50],[3,40],[4,0],[5,0]]
rev_cost = pd.DataFrame(df_list, columns = ['review_score', 'cost']) 

We apply the costs  for bad reviews to each review and drop the not needed columns:

In [4]:
reviews = reviews.merge(rev_cost, how='left', on='review_score'
                       ).drop(['review_comment_title', 'review_comment_message',
                              'review_creation_date','review_answer_timestamp'], axis=1)

In [5]:
reviews.head()

,review_id,order_id,review_score,cost
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,0
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,0
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,0
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,0
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,0


We merge the Orders with the review and seller data and drop not needed columns:

In [6]:
data = orders_details.merge(sellers, how='left', on='seller_id'
                           ).merge(orders, how='left', on='order_id'
                                  ).merge(reviews, how='left', on='order_id')

In [7]:
data = data.drop(['quantity', 'order_item_id', 'freight_value', 'delay_to_carrier',
                  'customer_id','order_approved_at','order_delivered_carrier_date',
                  'shipping_limit_date', 'order_status','order_delivered_customer_date',
                  'order_estimated_delivery_date', 'review_id'], axis=1)

Add a column for order date and calulate the Olist revenue (20% per booking value):

In [108]:
data['order_date'] = pd.to_datetime(data['order_purchase_timestamp'])
data['Olist_rev'] = data['price']*0.2

We set the Order Date as index so we can view per month

In [109]:
seller_review_data = data.set_index(['order_date'])
seller_review_data.head()
#data.head()

,order_id,product_id,seller_id,price,seller_city,seller_state,wait_time,share_one_stars,share_of_five_stars,avg_review_score,n_orders,order_purchase_timestamp,review_score,cost,Olist_rev
order_date,,,,,,,,,,,,,,,
2017-09-13 08:59:02,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,volta redonda,SP,14.145738,0.123188,0.536232,4.014493,138.0,2017-09-13 08:59:02,5,0,11.780
2017-04-26 10:53:06,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,sao paulo,SP,14.815091,0.155738,0.475410,3.827869,122.0,2017-04-26 10:53:06,4,0,47.980
2018-01-14 14:33:31,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,borda da mata,MG,11.863262,0.083333,0.333333,4.000000,12.0,2018-01-14 14:33:31,5,0,39.800
2018-08-08 10:00:35,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,franca,SP,12.984201,0.076923,0.692308,4.384615,13.0,2018-08-08 10:00:35,4,0,2.598
2017-02-04 13:57:51,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,loanda,PR,18.076695,0.142857,0.321429,3.678571,28.0,2017-02-04 13:57:51,5,0,39.980


We now group by seller for each month and add the monthly charged 80:

In [110]:
seller_review_data = seller_review_data.groupby(['seller_id', pd.Grouper(freq='M')]
                                               ).agg({'order_id':'nunique','price':'sum',
                                                      'wait_time':'first', 'avg_review_score':'first',
                                                      'n_orders':'first', 'Olist_rev':'sum', 'review_score':'mean',
                                                      'cost':'mean'}).fillna(0)

In [111]:
seller_review_data['Olist_rev'] = seller_review_data['Olist_rev']+80
df = seller_review_data.reset_index()

View some date for one seller

In [118]:
df.query("seller_id == '9d7a1d34a5052409006425275ba1c2b4'")

,seller_id,order_date,order_id,price,wait_time,avg_review_score,n_orders,Olist_rev,review_score,cost
10409,9d7a1d34a5052409006425275ba1c2b4,2017-07-31,2,678.60,12.984201,4.384615,13.0,215.720,1.800000,80.0
10410,9d7a1d34a5052409006425275ba1c2b4,2017-12-31,2,60.98,12.984201,4.384615,13.0,92.196,5.000000,0.0
10411,9d7a1d34a5052409006425275ba1c2b4,2018-02-28,3,38.49,12.984201,4.384615,13.0,87.698,5.000000,0.0
10412,9d7a1d34a5052409006425275ba1c2b4,2018-04-30,1,49.99,12.984201,4.384615,13.0,89.998,5.000000,0.0
10413,9d7a1d34a5052409006425275ba1c2b4,2018-06-30,2,191.98,12.984201,4.384615,13.0,118.396,4.000000,20.0
10414,9d7a1d34a5052409006425275ba1c2b4,2018-08-31,3,34.78,12.984201,4.384615,13.0,86.956,4.333333,0.0


# Find the threshold from where a seller should be banned from our platform

You can set the threshold manually and we look att the orders for the last 130 days

In [123]:
@interact
def plot_revs(treshhold=2.7):
    max_date = (df['order_date'].max() - timedelta(days=130))
    print('start_date:')
    print(max_date)
    df_ = df.query("order_date > @max_date")
    print('avr_revenue')
    print(df_['Olist_rev'].mean().round())
    
    df_ = df.query("review_score <" + str(treshhold))
    plt.figure(figsize=(12, 6))
    b = sns.distplot(df_['order_id'])
    b.set_xlabel("Number of Orders",fontsize=30)
    
    print('Nof_Sellers, avg_revenue ')
    print(df_['seller_id'].count(), ',' , (df_['Olist_rev'].mean()-df_['cost'].mean()).round())
    
    return df_['seller_id']
    

interactive(children=(FloatSlider(value=2.7, description='treshhold', max=8.100000000000001, min=-2.7), Output…